###ETL com Python

Base de dados: [Agentes/suspeitos identificados em crimes registados, segundo o escalão etário, por crime](https://dados.gov.pt/pt/datasets/agentes-suspeitos-identificados-em-crimes-registados-segundo-o-escalao-etario-por-crime/).

###Descrição:

Dimensões: Ano, tipo crime (nível 1), tipo crime (nível 2), tipo crime (nível 3), escalão etário.

Métricas: Número de intervenientes.

In [ ]:
!pip install azure-storage-blob

In [ ]:
!pip install ydata-profiling

In [3]:
from azure.storage.blob import BlobServiceClient, BlobClient, generate_blob_sas, BlobSasPermissions
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import requests
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.max_rows', None)

##Extract

Extraindo base de dados de um armazenamento de blob do azure.

In [ ]:
# Não esqueça de substituir com as suas informações!!!
account_name = 'account_name'
account_key = 'account_key'
container_name = 'container_name'
blob_name = 'blob_name'

# Definir validade SAS
sas_expiry = datetime.utcnow() + timedelta(hours=1)

# Permissões necessárias para a SAS (leitura, escrita)
permissions = BlobSasPermissions(read=True, write=True)

# Gerar a SAS para o blob
sas_token = generate_blob_sas(
    account_name=account_name,
    container_name=container_name,
    blob_name=blob_name,
    account_key=account_key,
    permission=permissions,
    expiry=sas_expiry
)

# Observação: Esse é um processo feito de forma manual, você pode gerar Token SAS dentro do ambiente azure!.
print("Token SAS para o blob:")
print(sas_token)

In [ ]:
blob_url = f"https://{account_name}.blob.core.windows.net/{container_name}/{blob_name}"
# Adicionar SAS ao URL do blob
sas_blob_url = f"{blob_url}?{sas_token}"
print("URL do blob com SAS:")
print(sas_blob_url)

In [7]:
sas_blob_url = "link da url sas" # <--- Substitua a informação.

## Análise DataFrame

Gerando um relatório para uma rápida análise.

In [8]:
df = pd.read_csv(sas_blob_url, delimiter=';')

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
# relatório do dataframe
profile = ProfileReport(df)
profile.to_file("relatorio.html")

## Transform

Transformando os dados!

In [12]:
df.drop('NumeroIntervenientes', axis=1, inplace=True)

In [13]:
filtro_total = df[['TipoCrimeN1',	'TipoCrimeN2',	'TipoCrimeN3',	'EscalaoEtario']].apply(lambda col: col.str.contains('Total', case=False))
linhas_com_total = filtro_total.all(axis=1)
df_filtrado = df[~linhas_com_total].replace('Total', np.nan)

In [ ]:
df_filtrado.info()

In [15]:
df_filtrado.to_csv('agentessuspeitosTV10.csv', index=False)

##Load

Carregando os dados transformados para o ambiente azure.

In [ ]:
local_file_path = "/content/agentessuspeitosTV10.csv"

# Realizando a requisição POST para enviar o arquivo
with open(local_file_path, "rb") as f:
    response = requests.put(sas_blob_url, data=f, headers={"x-ms-blob-type": "BlockBlob", "Content-Type": "application/octet-stream"})

if response.status_code == 201:
    print("Arquivo carregado com sucesso!")
else:
    print("Falha ao carregar o arquivo. Código de status:", response.status_code)